# Assignment 3 - Parameter Learning
## Probabilistic Models UE
### WS 2017

---
<div class="alert alert-warning">
**Due:** 21.11.2017, 11:00
</div>

<div class="alert alert-danger">
Please only submit the .ipynb file via Moodle!
</div>

<div class="alert alert-info">
**Hints:**  

<ul>
<li>You can use the `loadtxt` function to load data from text files, e.g.  
    ```
    data = loadtxt('train.txt', dtype=int)
    ```  
</li>
<li>
To count elements in a Numpy array, you can use the `bincount` function. Read the documentation for details, but in principle, it works like this:  

```
>>> a = array([0, 0, 1, 1, 0])
>>> print bincount(a, minlength=2)
[3 2]
```  

You can also just count elements in a loop, if you want.
</li>
</ul>

</div>

---
After watching hours of medical dramas on television, a medicine freshman tries to figure out the perfect prediction model for heart diseases.
Some of her computer science colleagues told her about how Bayesian networks can
be used for symptom diagnosis, so she decides to model her ideas using this
technique. In order to find out the (conditional) probability distributions, she needs to
find a lot of training examples. One of her computer science colleagues
cracks the computer system of the university clinic and copies a lot of medical
patient data. The data is available as "train.txt" in Moodle. 

Throughout the whole assignment assume that all variables are boolean (false=0 or true=1). The data for this assignment is stored in two text files, "train.txt" and "test.txt". They contain 500 (train) and 5000 (test) 
samples of the following 5 random variables: 

 - Column 0: S ... stress 
 - Column 1: E ... easily catches cold  
 - Column 2: G ... genetic disposition 
 - Column 3: I ... increased blood pressure
 - Column 4: H ... heart attack

---

# 1. Finding a Model for Strokes

<div class="alert alert-info">
**Hint:** The next three exercises are quite similar. It might be a good idea to first write reusable functions for estimating certain kinds of probability distributions.
</div>
## 1.1 Model 1 (6 points)

First, she decides to try out the following, very simple, model:

<img style='width:100%;  max-width:400px;' src="bn_mod1.svg">

 - **Which (conditional) probability tables are necessary to represent the Model 1?** Choose the finest factorisation possible - e.g., for a model $A \rightarrow B \rightarrow C$, you want to model $P(A)$, $P(B \mid A)$, and $P(C \mid B)$, and **not** just $P(A, B, C)$, or $P(A)$ and $P(B, C \mid A)$.

P(G) P(S) P(E) P(I) P(H)

 - **Estimate the probability tables (PDs) of Model 1.**

In [37]:
import numpy as np
import math
from math import exp, expm1

data = np.loadtxt('train.txt', dtype=int)
data_test = np.loadtxt('test.txt', dtype=int)
rows = data.shape[0]
rows = 500.0

In [38]:


P_S = sum(data[:,0])/rows
P_E = sum(data[:,1])/rows
P_G = sum(data[:,2])/rows
P_I = sum(data[:,3])/rows
P_H = sum(data[:,4])/rows

P_S = np.array([1-P_S, P_S])
P_E = np.array([1-P_E, P_E])
P_G = np.array([1-P_G, P_G])
P_I = np.array([1-P_I, P_I])
P_H = np.array([1-P_H, P_H])


 - **Compute the log-likelihood of the train data given Model 1.**

In [39]:
def logL(P_S,P_E,P_G,P_I,P_H,data,rows):
    cum = 0
    for i in range(0,int(rows)):
        cum = cum + math.log10(P_S[data[i,0]]*P_E[data[i,1]]*P_G[data[i,2]]*P_I[data[i,3]]*P_H[data[i,4]])
    return cum
logL(P_S,P_E,P_G,P_I,P_H,data,500)

-421.6182524066589

## 1.2 Model 2 (6 points)

Not satisfied with the results, our freshman decides that Model 1 is probably too simple to represent the real world. She adds some edges to the model, and tries again:

<img  style='width:100%;  max-width:400px;' src="bn_mod2.svg">

 - **Which (conditional) probability tables are necessary to represent Model 2?** Choose the finest factorisation possible - e.g., for a model $A \rightarrow B \rightarrow C$, you want to model $P(A)$, $P(B \mid A)$, and $P(C \mid B)$, and **not** just $P(A, B, C)$, or $P(A)$ and $P(B, C \mid A)$.

P(G), P(S), P(I|S,G), P(E|S),P(H|I)

 - **Estimate the probability tables (PDs) of Model 2.**

In [40]:
#P(G) and P(S) are priors therefore they will be the same as in 1.1
               #GS
count_Evid_of_I_00 = 0.0
count_Evid_of_I_01 = 0.0
count_Evid_of_I_10 = 0.0
count_Evid_of_I_11 = 0.0

count_I_given_00 = 0.0
count_I_given_01 = 0.0
count_I_given_10 = 0.0
count_I_given_11 = 0.0

count_H_given_0 = 0.0
count_H_given_1 = 0.0

count_E_given_0 = 0.0
count_E_given_1 = 0.0
#compute I
for i in range(0,int(rows)):
    #computing I #00
    if data[i,2] == 0 and data[i,0] == 0:
        count_Evid_of_I_00 = count_Evid_of_I_00+1
        if data[i,3] == 1:
            count_I_given_00 = count_I_given_00 + 1
    #11
    if data[i,2] == 1 and data[i,0] == 1:
        count_Evid_of_I_11 = count_Evid_of_I_11+1
        if data[i,3] == 1:
            count_I_given_11 = count_I_given_11 + 1
     #10       
    if data[i,2] == 1 and data[i,0] == 0:
        count_Evid_of_I_10 = count_Evid_of_I_10+1
        if data[i,3] == 1:
            count_I_given_10 = count_I_given_10 + 1
    #01
    if data[i,2] == 0 and data[i,0] == 1:
        count_Evid_of_I_01 = count_Evid_of_I_01+1
        if data[i,3] == 1:
            count_I_given_01 = count_I_given_01 + 1
    #compute H
    if data[i,3] == 0 and data[i,4] == 1:
        count_H_given_0 = count_H_given_0+1
    
    if data[i,3] == 1 and data[i,4] == 1:
        count_H_given_1 = count_H_given_1+1
        
    #compute E
    if data[i,0] == 0 and data[i,1] == 1:
        count_E_given_0 = count_E_given_0+1
    
    if data[i,0] == 1 and data[i,1] == 1:
        count_E_given_1 = count_E_given_1+1
        

P_I_bayes_m1 = np.array([
               [1-(count_I_given_00/count_Evid_of_I_00),(count_I_given_00/count_Evid_of_I_00)], #G0S0
               [1-(count_I_given_10/count_Evid_of_I_10),(count_I_given_10/count_Evid_of_I_10)],  #G1S0
               [1-(count_I_given_01/count_Evid_of_I_01),(count_I_given_01/count_Evid_of_I_01)],  #G0S1
               [1-(count_I_given_11/count_Evid_of_I_11),(count_I_given_11/count_Evid_of_I_11)]]) #G1S1

P_H_bayes_m1 = np.array([
                     [1-(count_H_given_0/((P_I[0])*rows)),(count_H_given_0/((P_I[0])*rows))], #I0
                     [1-(count_H_given_1/(P_I[1]*rows)),(count_H_given_1/(P_I[1]*rows))]          #I1
                    ]) 

P_E_bayes_m1 = np.array([
                     [1-(count_E_given_0/((P_S[0])*rows)),(count_E_given_0/((P_S[0])*rows))], #S0
                     [1-(count_E_given_1/(P_S[1]*rows)),(count_E_given_1/(P_S[1]*rows))]      #S1
                    ]) 


 - **Compute the log-likelihood of the train data given Model 2.**

In [41]:
def logL1(P_S,P_E,P_G,P_I,P_H,data,rows):
    cum = 0
    for i in range(0,int(rows)):
        if data[i,0] == 0 and data[i,2] == 0:
            access = 0
        elif data[i,0] == 0 and data[i,2] == 1:
            access = 1
        elif data[i,0] == 1 and data[i,2] == 0:
            access = 2
        elif data[i,0] == 1 and data[i,2] == 1:
            access = 3
        cum = cum + math.log10(P_S[data[i,0]]*P_E[data[i,0]][data[i,1]]*P_G[data[i,2]]*P_I[access][data[i,3]]*P_H[data[i,3]][data[i,4]])                                     
    return cum
logL1(P_S,P_E_bayes_m1,P_G,P_I_bayes_m1,P_H_bayes_m1,data,500)

-408.56817038870247

## 1.2 Model 3 (6 points)

Finally, she decides to try out an even more complex model:

<img  style='width:100%;  max-width:400px;' src="bn_mod3.svg">

 - **Which (conditional) probability tables are necessary to represent Model 3?** Choose the finest factorisation possible - e.g., for a model $A \rightarrow B \rightarrow C$, you want to model $P(A)$, $P(B \mid A)$, and $P(C \mid B)$, and **not** just $P(A, B, C)$, or $P(A)$ and $P(B, C \mid A)$.

p(G),p(S|G),P(I|G,S),P(E|S,I),P(H|E,I)

 - **Estimate the probability tables (PDs) of Model 3.**

In [42]:
#P(G) is still the same, however p(S) changed as its dependent on G in the upper mentioned model
count_S_given_0 = 0.0
count_S_given_1 = 0.0

count_Evid_of_I_00 = 0.0
count_Evid_of_I_10 = 0.0
count_Evid_of_I_01 = 0.0
count_Evid_of_I_11 = 0.0

count_I_given_00 = 0.0
count_I_given_10 = 0.0
count_I_given_01 = 0.0
count_I_given_11 = 0.0

count_Evid_of_E_00 = 0.0
count_Evid_of_E_10 = 0.0
count_Evid_of_E_01 = 0.0
count_Evid_of_E_11 = 0.0

count_E_given_00 = 0.0
count_E_given_10 = 0.0
count_E_given_01 = 0.0
count_E_given_11 = 0.0

count_Evid_of_H_00 = 0.0
count_Evid_of_H_10 = 0.0
count_Evid_of_H_01 = 0.0
count_Evid_of_H_11 = 0.0

count_H_given_00 = 0.0
count_H_given_10 = 0.0
count_H_given_01 = 0.0
count_H_given_11 = 0.0

count_S_given_0 = 0.0
count_S_given_1 = 0.0

for i in range(0,int(rows)):
   
        
    #compute I
    #computing I #00
    if data[i,2] == 0 and data[i,0] == 0:
        count_Evid_of_I_00 = count_Evid_of_I_00+1
        if data[i,3] == 1:
            count_I_given_00 = count_I_given_00 + 1
    #11
    if data[i,2] == 1 and data[i,0] == 1:
        count_Evid_of_I_11 = count_Evid_of_I_11+1
        if data[i,3] == 1:
            count_I_given_11 = count_I_given_11 + 1
     #10       
    if data[i,2] == 1 and data[i,0] == 0:
        count_Evid_of_I_10 = count_Evid_of_I_10+1
        if data[i,3] == 1:
            count_I_given_10 = count_I_given_10 + 1
    #01
    if data[i,2] == 0 and data[i,0] == 1:
        count_Evid_of_I_01 = count_Evid_of_I_01+1
        if data[i,3] == 1:
            count_I_given_01 = count_I_given_01 + 1
            
    #compute E
    #computing E #00
    if data[i,0] == 0 and data[i,3] == 0:
        count_Evid_of_E_00 = count_Evid_of_E_00+1
        if data[i,1] == 1:
            count_E_given_00 = count_E_given_00 + 1
    #11
    if data[i,0] == 1 and data[i,3] == 1:
        count_Evid_of_E_11 = count_Evid_of_E_11+1
        if data[i,1] == 1:
            count_E_given_11 = count_E_given_11 + 1
     #10       
    if data[i,0] == 1 and data[i,3] == 0:
        count_Evid_of_E_10 = count_Evid_of_E_10+1
        if data[i,1] == 1:
            count_E_given_10 = count_E_given_10 + 1
    #01
    if data[i,0] == 0 and data[i,3] == 1:
        count_Evid_of_E_01 = count_Evid_of_E_01+1
        if data[i,1] == 1:
            count_E_given_01 = count_E_given_01 + 1
    
    
    #compute H
    #computing H #00
    if data[i,1] == 0 and data[i,3] == 0:
        count_Evid_of_H_00 = count_Evid_of_H_00+1
        if data[i,4] == 1:
            count_H_given_00 = count_H_given_00 + 1
    #11
    if data[i,1] == 1 and data[i,3] == 1:
        count_Evid_of_H_11 = count_Evid_of_H_11+1
        if data[i,4] == 1:
            count_H_given_11 = count_H_given_11 + 1
     #10       
    if data[i,1] == 1 and data[i,3] == 0:
        count_Evid_of_H_10 = count_Evid_of_H_10+1
        if data[i,4] == 1:
            count_H_given_10 = count_H_given_10 + 1
    #01
    if data[i,1] == 0 and data[i,3] == 1:
        count_Evid_of_H_01 = count_Evid_of_H_01+1
        if data[i,4] == 1:
            count_H_given_01 = count_H_given_01 + 1
    
    #compute S
    if data[i,2] == 0 and data[i,0] == 1:
        count_S_given_0 = count_S_given_0+1
    
    if data[i,2] == 1 and data[i,0] == 1:
        count_S_given_1 = count_S_given_1+1
            
P_I_bayes_m2 = np.array([
               [1-(count_I_given_00/count_Evid_of_I_00),(count_I_given_00/count_Evid_of_I_00)], #G0S0
               [1-(count_I_given_10/count_Evid_of_I_10),(count_I_given_10/count_Evid_of_I_10)],  #G1S0
               [1-(count_I_given_01/count_Evid_of_I_01),(count_I_given_01/count_Evid_of_I_01)],  #G0S1
               [1-(count_I_given_11/count_Evid_of_I_11),(count_I_given_11/count_Evid_of_I_11)]]) #G1S1
P_E_bayes_m2 = np.array([
               [1-(count_E_given_00/count_Evid_of_E_00),(count_E_given_00/count_Evid_of_E_00)], #S0I0
               [1-(count_E_given_10/count_Evid_of_E_10),(count_E_given_10/count_Evid_of_E_10)], #S1I0
               [1-(count_E_given_01/count_Evid_of_E_01),(count_E_given_01/count_Evid_of_E_01)],  #S0I1
               [1-(count_E_given_11/count_Evid_of_E_11),(count_E_given_11/count_Evid_of_E_11)]]) #S1I1
P_H_bayes_m2 = np.array([
               [1-(count_H_given_00/count_Evid_of_H_00),(count_H_given_00/count_Evid_of_H_00)], #E0I0
               [1-(count_H_given_10/count_Evid_of_H_10),(count_H_given_10/count_Evid_of_H_10)],  #E1I0
               [1-(count_H_given_01/count_Evid_of_H_01),(count_H_given_01/count_Evid_of_H_01)],  #E0I1
               [1-(count_H_given_11/count_Evid_of_H_11),(count_H_given_11/count_Evid_of_H_11)]]) #E1I1

P_S_bayes_m2 = np.array([
                     [1-(count_S_given_0/((P_G[0])*rows)),(count_S_given_0/((P_G[0])*rows))], #G0
                     [1-(count_S_given_1/(P_G[1]*rows)),(count_S_given_1/(P_G[1]*rows))]          #G1
                    ]) 


 - **Compute the log-likelihood of the train data given Model 3.**

In [43]:
def logL2(P_S,P_E,P_G,P_I,P_H,data,rows):
    cum = 0
    for i in range(0,int(rows)):
        if data[i,0] == 0 and data[i,2] == 0:
            access = 0
        elif data[i,0] == 0 and data[i,2] == 1:
            access = 1
        elif data[i,0] == 1 and data[i,2] == 0:
            access = 2
        elif data[i,0] == 1 and data[i,2] == 1:
            access = 3
            
        if data[i,0] == 0 and data[i,3] == 0:
            access2 = 0
        elif data[i,0] == 1 and data[i,3] == 0:
            access2 = 1
        elif data[i,0] == 0 and data[i,3] == 1:
            access2 = 2
        elif data[i,0] == 1 and data[i,3] == 1:
            access2 = 3
            
        if data[i,1] == 0 and data[i,3] == 0:
            access3 = 0
        elif data[i,1] == 1 and data[i,3] == 0:
            access3 = 1
        elif data[i,1] == 0 and data[i,3] == 1:
            access3 = 2
        elif data[i,1] == 1 and data[i,3] == 1:
            access3 = 3
        cum = cum + math.log10((P_S[data[i,2]][data[i,0]])*P_E[access2][data[i,1]]*P_G[data[i,2]]*P_I[access][data[i,3]]*P_H[access3][data[i,4]])                                     
    return cum
logL2(P_S_bayes_m2,P_E_bayes_m2,P_G,P_I_bayes_m2,P_H_bayes_m2,data,500)

-408.12130459532096

---

# 2. Comparison

## 2.1 Compare Train Log-Likelihoods (3 points)

Compare the log-likelihoods of the training data given the three models. Why are they different? What does it **mean** that the data has a higher log-likelihood for a model?


Model 1 Log-likelihood:-421.618252407
Model 2 Log-likelihood:-408.568170389
Model 3 Log-likelihood:-408.121304595

These results show that the training data is most likely to be generated from model 3 than model 2 followed by model 1.

They are different due to the architecture of the bayesian net that corresponds to each model. Model 3 adds more dependencies as a way to fit more precisely the given training data whereas model 1 assumes no dependancies-underfittign- and therefore, it has a very low likelihood compared to the other 2 models.

The data has a higher likelihood for a model means that this model -in this case, model 3- has a higher probability of generating these data -training data-.

## 2.2 Compare Test Log-Likelihoods (3 points)

The computer science colleague manages to obtain more data from the clinic's network and stores it in the file `test.txt`. Our medicine freshman is eager to try the models on the new data.

Compute the log-likelihood of the test data for each of the three models. **Do not learn probability tables!** Use the probability tables you computed in 1).

**What is the difference compared to the log-likelihoods of the training dataset** (ignore the magnitude difference due to different number of data points in each set)? Explain the difference! From your results, which of the models would you select, and why?

In [48]:
print(logL(P_S,P_E,P_G,P_I,P_H,data_test,5000))
print(logL1(P_S,P_E_bayes_m1,P_G,P_I_bayes_m1,P_H_bayes_m1,data_test,5000.0))
print(logL2(P_S_bayes_m2,P_E_bayes_m2,P_G,P_I_bayes_m2,P_H_bayes_m2,data_test,5000.0))

-4145.54025994
-4002.84858896
-4007.19714656



Model 1 Log-likelihood:-4145.54025994
Model 2 Log-likelihood:-4002.84858896
Model 3 Log-likelihood:-4007.19714656
Its clearly shown that model 3 overfitts that training data and therefore it failed to generalize on the testing data as compared to model 2 that shows a better likelihood. ofcourse model 1 has a lower value than the rest due to the fact that it underfitts.

Model 2 would be selected as it has the higher likelihood on the test data which means that it was more capable of generalizing on the 